# Generate precomputed genre jear json

### Libraries

In [ ]:
import pandas as pd
import pickle
import plotly
import plotly.graph_objs as go
import plotly.offline as pyo
import plotly.express as px
pyo.init_notebook_mode()
import os.path
import json

In [1]:
plotly.__version__ # I got 4.5.0

NameError: name 'plotly' is not defined

In [ ]:
pd.__version__

### Import

In [4]:
import_data = pickle.load(open('song_data_20200304.pickle', "rb"))

In [5]:
import_data.head(2)

,title,artist,release_year,track_id,1999,2000,2001,2002,2003,2004,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,Bohemian Rhapsody,Queen,1975,7tFiyTwD0nx5a1eklYtX2J,1,1,1,1,1,1,...,0.28800,0.392,0.402,0.000000,0.2430,-9.961,0.0536,143.883,0.228,Rock
1,Hotel California,Eagles,1977,40riOy7x9W7GXjyGp4pjAv,2,4,3,3,2,2,...,0.00574,0.579,0.508,0.000494,0.0575,-9.484,0.0270,147.125,0.609,Classic rock


# Plot with genres

In [6]:
year = str(2015)
feature = "danceability"
color = "genre"
fig = px.scatter(import_data.dropna(subset = [year]), x=year, y=feature, color=color, color_discrete_sequence=px.colors.qualitative.Dark24)

# fig.update_traces(marker=dict(size=12,
#                               line=dict(width=2,
#                                         color='DarkSlateGrey')),
#                   selector=dict(mode='markers'))
fig.show()

In [7]:
#fig.write_image("plotly_track_graph.png")

In [8]:
import_data.head()

,title,artist,release_year,track_id,1999,2000,2001,2002,2003,2004,...,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,genre
0,Bohemian Rhapsody,Queen,1975,7tFiyTwD0nx5a1eklYtX2J,1,1,1,1,1,1,...,0.28800,0.392,0.402,0.000000,0.2430,-9.961,0.0536,143.883,0.228,Rock
1,Hotel California,Eagles,1977,40riOy7x9W7GXjyGp4pjAv,2,4,3,3,2,2,...,0.00574,0.579,0.508,0.000494,0.0575,-9.484,0.0270,147.125,0.609,Classic rock
2,Child In Time,Deep Purple,1972,5TrfusgKC1BYic5LaWCoKm,3,2,2,2,3,3,...,0.17000,0.338,0.461,0.564000,0.0927,-10.740,0.0396,126.698,0.351,Classic rock
3,Stairway To Heaven,Led Zeppelin,1971,5CQ30WqJwcep0pYcV4AMNc,4,3,4,4,4,4,...,0.58000,0.338,0.340,0.003200,0.1160,-12.049,0.0339,82.433,0.197,Classic rock
4,Paradise By The Dashboard Light,Meat Loaf,1978,2g7gviEeJr6pyxO7G35EWQ,5,5,5,5,5,6,...,0.21400,0.326,0.872,0.000000,0.4610,-7.602,0.1220,173.381,0.492,Soft rock


In [9]:
import_data.columns

Index(['title', 'artist', 'release_year', 'track_id', '1999', '2000', '2001',
       '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       'artist_id', 'album_id', 'image_640', 'image_64', 'popularity',
       'duration', 'preview_url', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence', 'genre'],
      dtype='object')

# MAKE GENRE DATA

## Data for Scatterplot

In [10]:
years = ['1999', '2000', '2001', '2002',
       '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010', '2011',
       '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
required_means = ['release_year', 'popularity',
       'duration', 'acousticness', 'danceability', 'energy',
       'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo',
       'valence']

In [11]:
aggregation_fun = {column : 'mean' for column in required_means}

In [12]:

print(*aggregation_fun.items(), sep='\n')

('release_year', 'mean')
('popularity', 'mean')
('duration', 'mean')
('acousticness', 'mean')
('danceability', 'mean')
('energy', 'mean')
('instrumentalness', 'mean')
('liveness', 'mean')
('loudness', 'mean')
('speechiness', 'mean')
('tempo', 'mean')
('valence', 'mean')


# Scatterplot separate files per year

In [13]:
for year in years:
    #only take tracks present in this year's top 2000
    year_data = import_data.dropna(subset = [year])
    
    #get means and size
    year_means = year_data.groupby("genre").agg(aggregation_fun)
    year_genre_size= year_data.groupby("genre").size().rename("size")
    
    # #combine
    year_output = pd.concat([year_genre_size,year_means], axis = 1)
    
    #save
    year_output.to_json(os.path.join("data", year + '_genres_index.json'), orient='index', indent=4)
    year_output.reset_index().to_json(os.path.join("data", year + '_genres.json'), orient='records', indent=4)
    year_output.to_csv(os.path.join("data", year+'_genres.csv'), index = True)
    # year_output.to_pickle(os.path.join("data", year+'_genres.pickle'))

In [14]:
year_output

,size,release_year,popularity,duration,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence
genre,,,,,,,,,,,,,
Chanson,4,1970.000000,54.250000,202096.250000,0.460750,0.372500,0.528000,6.300000e-07,0.320500,-8.349500,0.052850,125.362250,0.537500
Classic rock,502,1976.007968,59.350598,274653.758964,0.312339,0.522267,0.572853,6.333379e-02,0.190404,-10.209612,0.047435,120.746592,0.545516
Country,11,1984.636364,58.454545,196634.818182,0.490909,0.551909,0.504182,9.126290e-03,0.298327,-9.550273,0.095791,105.330727,0.606727
Disco,4,1982.500000,56.500000,232306.250000,0.207350,0.758000,0.754000,7.907500e-05,0.327975,-9.089000,0.078700,130.267000,0.765000
Dutch rock,265,1995.664151,34.305660,240690.147170,0.335401,0.514517,0.560249,1.653146e-02,0.277076,-9.446645,0.048770,118.709008,0.464363
Electronica/dance,6,1994.666667,43.333333,264933.000000,0.017973,0.482833,0.814500,3.365667e-01,0.233850,-7.017833,0.051133,116.496833,0.260117
Folk,6,1982.666667,49.000000,224079.666667,0.579885,0.380667,0.491500,1.560183e-04,0.231683,-9.107667,0.079317,127.342000,0.411667
Hip-hop,8,2001.875000,57.500000,247770.750000,0.255611,0.591250,0.621375,2.994409e-01,0.203613,-9.463250,0.036350,116.760500,0.413375
House,7,2003.000000,55.000000,260254.857143,0.017757,0.677857,0.783286,1.758976e-01,0.192686,-6.922000,0.087143,124.346714,0.412857


# One combinded file for scatterplot (request Rens)

In [23]:
scatterplot = dict()

for year in years[:1]:
    #only take tracks present in this year's top 2000
    year_data = import_data.dropna(subset = [year])
    
    #get means and size
    year_means = year_data.groupby("genre").agg(aggregation_fun)
    year_genre_size= year_data.groupby("genre").size().rename("size")
    
    # combine
    year_output = pd.concat([year_means,year_genre_size], axis = 1)
    year_output = year_output.reset_index()

    # add each year as an entry of a dict to get a nested dict as end result
    scatterplot[year] = year_output.to_dict(orient="records")
    
# save
with open(os.path.join("data","scatterplot.json"), "w") as scatterjson:
    json.dump(scatterplot,scatterjson, indent=4)

In [19]:
year_output.reset_index()

,genre,release_year,popularity,duration,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo,valence,size
0,Blues,1969.333333,38.333333,221341.666667,0.286867,0.669000,0.629000,0.003243,0.224900,-9.111000,0.055467,104.820333,0.619667,3
1,Chanson,1966.636364,45.090909,195842.090909,0.615273,0.418545,0.393000,0.009648,0.250636,-10.759091,0.043064,113.131455,0.506545,11
2,Classic rock,1972.997955,55.952965,243444.955010,0.346013,0.532777,0.570620,0.048460,0.183377,-10.198301,0.048185,121.107313,0.592672,489
3,Country,1973.400000,53.600000,182439.800000,0.707400,0.567200,0.395800,0.000253,0.332600,-11.202800,0.067540,110.781400,0.688600,5
4,Disco,1980.500000,47.909091,275237.636364,0.184352,0.710773,0.726000,0.088921,0.203595,-9.290318,0.063327,118.479864,0.745500,22
5,Dutch rock,1982.059322,28.135593,237274.279661,0.384832,0.531017,0.540551,0.010701,0.292166,-10.828136,0.055925,115.068992,0.513283,118
6,Folk,1967.500000,34.312500,208656.375000,0.613957,0.469813,0.432462,0.007458,0.213881,-10.657500,0.054100,119.679250,0.553188,16
7,House,1978.000000,6.000000,199863.000000,0.007050,0.658000,0.808000,0.332000,0.255000,-8.546000,0.034900,129.983000,0.298000,1
8,Jazz,1966.727273,43.500000,219701.045455,0.644441,0.499864,0.373132,0.059549,0.250245,-13.371136,0.047473,114.520364,0.501850,22
9,Karaoke,1972.294118,1.705882,203753.529412,0.279428,0.565059,0.443412,0.613465,0.175818,-12.673647,0.039729,127.016529,0.541176,17


# Data for line chart

In [35]:
lineplot = pd.DataFrame(columns=required_means)
for year in years:
    #only take tracks present in this year's top 2000
    year_data = import_data.dropna(subset = [year])
    
    #get means and size
    year_means = year_data[required_means].mean() #get means
    year_means = year_means.rename(year)
    lineplot = lineplot.append(year_means)
lineplot.to_json(os.path.join("data", 'lineplot.json'), orient='index', indent=4)
#lineplot.transpose().to_json(os.path.join("data", 'lineplot_transpose.json'), orient='index', indent=4)

In [36]:
lineplot_object = list()
for feature in required_means:
    data = dict()
    data["key"] = feature
    year_list = list()
    for row in lineplot.iterrows():
        year_object = dict({"year":row[0],"data":row[1][feature]})
        year_list.append(year_object)
    data["data"] = year_list

    lineplot_object.append(data)

In [37]:
with open(os.path.join("data","lineplot_object.json"), "w") as lineobject:
    lineobject.write(json.dumps(lineplot_object, indent=4))